## Figure 7: Climate uncertainty

In [ ]:
# spatial libraries
import shapely.geometry
import rioxarray as rioxr
import geopandas as gpd
import xarray as xr
import xesmf as xe
import regionmask

from xclim.indicators import atmos
from xclim import core 

from tqdm.notebook import tqdm
import pandas as pd
import numpy as np
import cftime
import os

# plotting libraries
from plotly.subplots import make_subplots
import plotly.graph_objects as go
import plotly.express as px

# colors
cl = px.colors.qualitative.D3
cs = px.colors.sequential.Sunset

os.chdir('/home/rooda/Dropbox/Patagonia/')

## Grid reference

In [ ]:
# high resolution grid
raster_hr   = xr.open_dataset("/home/rooda/OGGM_results/PMET_OGGM_1980_2019m.nc").prcp

# DEMs to downscale temperature
dem_005 = xr.open_dataset("/home/rooda/OGGM_results/PMET_OGGM_1980_2019m.nc").hgt

dem_010 = xr.open_dataset("/home/rooda/OGGM_results/MSWEP_OGGM_1980_2019m.nc").hgt
regridder  = xe.Regridder(dem_010,   dem_005, "nearest_s2d")
dem_010    = regridder(dem_010)

dem_025 = xr.open_dataset("/home/rooda/OGGM_results/ERA5_OGGM_1980_2019m.nc").hgt
regridder  = xe.Regridder(dem_025,   dem_005, "nearest_s2d")
dem_025    = regridder(dem_025)

# lapse rate
lapse_rate = 0.0065 

## Preprocessing bias corrected data

In [ ]:
os.chdir("/home/rooda/OGGM_results/Future_climate_bc/")

baseline_period = slice("1980-01-01", "2010-01-01")
future_period   = slice("2070-01-01", "2100-01-01")

lat_coords = np.arange(-56,-40, 0.05)
lon_coords = np.arange(-76,-67, 0.05)

climate_list  = ["PMET", "ERA5", "CR2MET", "MSWEP"]
gcm_list  = ["ACCESS-CM2", "BCC-CSM2-MR", "CMCC-ESM2", "FGOALS-f3-L", "GFDL-ESM4", "CMCC-CM2-SR5", "KACE-1-0-G", "MPI-ESM1-2-HR", "MRI-ESM2-0", "MIROC6"]
ssp_list  = ['ssp126', 'ssp245', 'ssp370', 'ssp585']
bcm_list  = ['MVA', 'DQM', 'MBC']

In [ ]:
dataset_pp   = []
dataset_prsn = []
dataset_ppd  = []
dataset_t2m  = []

for climate in tqdm(climate_list):
    
    dataset_pp_climate   = []
    dataset_prsn_climate = []
    dataset_ppd_climate  = []
    dataset_t2m_climate  = []
    
    for gcm in tqdm(gcm_list, leave = False):
        for ssp in tqdm(ssp_list, leave = False):
            for bcm in bcm_list: 
        
                # pp and t2m in target resolution
                pp_projection = xr.open_dataset("PP_" + climate + "_" + gcm + "_" + ssp + "_"  + bcm + ".nc").pr
                pp_projection = pp_projection.sel(time = future_period)
                regridder     = xe.Regridder(pp_projection,   raster_hr, "nearest_s2d")
                pp_projection = regridder(pp_projection) ## ESMF.RegridMethod.NEAREST_STOD
                pp_projection.attrs['units']  = "kg m-2 s-1"
                pp_projection.attrs['standard_name']  = "precipitation_flux"
                
                t2m_projection = xr.open_dataset("T2M_" + climate + "_" + gcm + "_" + ssp + "_"  + bcm + ".nc").tas
                t2m_projection = t2m_projection.sel(time = future_period)
                t2m_projection = core.units.convert_units_to(t2m_projection, target = 'C')
                
                if climate == "ERA5": # uses lapse rate
                    regridder  = xe.Regridder(t2m_projection, raster_hr, "nearest_s2d")
                    t2m_projection   = regridder(t2m_projection) 
                    factor     = (dem_025 - dem_005)*lapse_rate
                    t2m_projection   =  t2m_projection + factor
                    
                if climate == "MSWEP": # uses lapse rate
                    regridder  = xe.Regridder(t2m_projection,   raster_hr, "nearest_s2d")
                    t2m_projection  = regridder(t2m_projection)
                    factor     = (dem_010 - dem_005)*lapse_rate
                    t2m_projection  =  t2m_projection + factor
                    
                if climate == "CR2MET" or climate == "PMET":  # same res, different grid
                    regridder  = xe.Regridder(t2m_projection, raster_hr, "nearest_s2d")
                    t2m_projection = regridder(t2m_projection) # simple case (same resolution)
                
                t2m_projection.attrs['units']  = "C"
                t2m_projection.attrs['standard_name']  = "air_temperature"
                
                # calculation of ppd (-1ºC threshold)
                ppd_projection = t2m_projection.where(t2m_projection >= -1)
                ppd_projection = (ppd_projection + 1).resample(time = "YS").sum(skipna = False)
                ppd_projection = ppd_projection.where(ppd_projection > 0)
                ppd_projection = ppd_projection.mean(dim="time")
                ppd_projection = ppd_projection.assign_coords({"CLIMATE": climate, "GCM": gcm, "SSP": ssp, "BCM": bcm})
                ppd_projection = ppd_projection.expand_dims(['CLIMATE', 'GCM', 'SSP', 'BCM'])
                dataset_ppd_climate.append(ppd_projection)    
                
                # calculation of prsn (brown method)
                prsn_projection = atmos.snowfall_approximation(pp_projection, t2m_projection, method='brown', thresh='0 degC')
                prsn_projection = core.units.convert_units_to(prsn_projection, target = 'mm month-1', context = "hydro")
                prsn_projection = prsn_projection.resample(time = "YS").sum(skipna = False)
                prsn_projection = prsn_projection.mean(dim="time")
                prsn_projection = prsn_projection.assign_coords({"CLIMATE": climate, "GCM": gcm, "SSP": ssp, "BCM": bcm})
                prsn_projection = prsn_projection.expand_dims(['CLIMATE', 'GCM', 'SSP', 'BCM'])
                dataset_prsn_climate.append(prsn_projection) 
                
                # calculation of air temperature
                t2m_projection = t2m_projection.resample(time = "YS").mean(skipna = False)
                t2m_projection = t2m_projection.mean(dim="time")
                t2m_projection = t2m_projection.assign_coords({"CLIMATE": climate, "GCM": gcm, "SSP": ssp, "BCM": bcm})
                t2m_projection = t2m_projection.expand_dims(['CLIMATE', 'GCM', 'SSP', 'BCM'])
                dataset_t2m_climate.append(t2m_projection)               
                
                # calculation of total precipitation
                pp_projection = core.units.convert_units_to(pp_projection, target = 'mm month-1', context = "hydro")
                pp_projection = pp_projection.resample(time = "YS").sum(skipna = False)
                pp_projection = pp_projection.mean(dim="time")
                pp_projection = pp_projection.assign_coords({"CLIMATE": climate, "GCM": gcm, "SSP": ssp, "BCM": bcm})
                pp_projection = pp_projection.expand_dims(['CLIMATE', 'GCM', 'SSP', 'BCM'])
                dataset_pp_climate.append(pp_projection)
            
    # trick due to RAM limitation
    dataset_pp_climate = xr.combine_by_coords(dataset_pp_climate).load()
    dataset_pp.append(dataset_pp_climate)
                
    dataset_t2m_climate = xr.combine_by_coords(dataset_t2m_climate).load()
    dataset_t2m.append(dataset_t2m_climate)

    dataset_ppd_climate = xr.combine_by_coords(dataset_ppd_climate).load()
    dataset_ppd.append(dataset_ppd_climate)
                
    dataset_prsn_climate = xr.combine_by_coords(dataset_prsn_climate).load()
    dataset_prsn.append(dataset_prsn_climate)

dataset_pp  = xr.combine_by_coords(dataset_pp)
dataset_pp.to_netcdf("/home/rooda/OGGM_results/Future_climate_bc_concat/PP_projections_bc.nc")    

dataset_t2m  = xr.combine_by_coords(dataset_t2m)
dataset_t2m.to_netcdf("/home/rooda/OGGM_results/Future_climate_bc_concat/T2M_projections_bc.nc")

dataset_ppd  = xr.combine_by_coords(dataset_ppd)
dataset_ppd.to_netcdf("/home/rooda/OGGM_results/Future_climate_bc_concat/PPD_projections_bc.nc")

dataset_prsn  = xr.combine_by_coords(dataset_prsn)
dataset_prsn.to_netcdf("/home/rooda/OGGM_results/Future_climate_bc_concat/PRSN_projections_bc.nc")

## Data already processed

In [ ]:
# Catchment shapefiles
basins = gpd.read_file("GIS South/Basins_Patagonia_ice.shp")
basins = basins.set_index("ID")

names = ["Yelcho", "Baker", "Santa Cruz                           ", "Palena", "Grey", "Puelo", "Cisnes", "Aysen", "Pascua"]
basins.loc[basins.basin_area > 5000, "Name"] = names

# Glacier shapefiles
rgi6 = gpd.read_file("GIS South/Glaciers/RGI6_v2.shp")[["geometry"]]
rgi7 = gpd.read_file("GIS South/Glaciers/RGI7_v2.shp")[["geometry"]]
glaciers  = pd.concat([rgi6.geometry, rgi7.geometry])
glaciers  = glaciers.buffer(0.05) # mask to use for baseline climate

# Climate data
dataset_pp   = xr.open_dataset("/home/rooda/OGGM_results/Future_climate_bc_concat/PP_projections_bc.nc").to_array()[0]
dataset_t2m  = xr.open_dataset("/home/rooda/OGGM_results/Future_climate_bc_concat/T2M_projections_bc.nc").to_array()[0]
dataset_prsn = xr.open_dataset("/home/rooda/OGGM_results/Future_climate_bc_concat/PRSN_projections_bc.nc").to_array()[0]
dataset_ppd  = xr.open_dataset("/home/rooda/OGGM_results/Future_climate_bc_concat/PPD_projections_bc.nc").to_array()[0]

# mask: only glaciarated area
mask    = regionmask.mask_geopandas(glaciers, dataset_pp)   >= 0

dataset_pp    = dataset_pp.where(mask, drop = True)
dataset_t2m   = dataset_t2m.where(mask, drop = True)
dataset_prsn  = dataset_prsn.where(mask, drop = True)
dataset_ppd   = dataset_ppd.where(mask, drop = True) * 30 # daily

In [ ]:
dataset_pp_ensemble_std   = dataset_pp.std(dim = ["CLIMATE","GCM", "SSP", "BCM"])
dataset_t2m_ensemble_std  = dataset_t2m.std(dim = ["CLIMATE","GCM", "SSP", "BCM"])
dataset_pp_ensemble_mean   = dataset_pp.mean(dim = ["CLIMATE","GCM", "SSP", "BCM"])
dataset_t2m_ensemble_mean  = dataset_t2m.mean(dim = ["CLIMATE","GCM", "SSP", "BCM"])

dataset_pp_climate    = dataset_pp.std(dim = ["CLIMATE"]).mean(dim = ["GCM", "SSP", "BCM"])
dataset_t2m_climate   = dataset_t2m.std(dim = ["CLIMATE"]).mean(dim = ["GCM", "SSP", "BCM"])
dataset_prsn_climate  = dataset_prsn.std(dim = ["CLIMATE"]).mean(dim = ["GCM", "SSP", "BCM"])
dataset_ppd_climate   = dataset_ppd.std(dim = ["CLIMATE"]).mean(dim = ["GCM", "SSP", "BCM"])

dataset_pp_gcm    = dataset_pp.std(dim = ["GCM"]).mean(dim = ["CLIMATE", "SSP", "BCM"])
dataset_t2m_gcm   = dataset_t2m.std(dim = ["GCM"]).mean(dim = ["CLIMATE", "SSP", "BCM"])
dataset_prsn_gcm  = dataset_prsn.std(dim = ["GCM"]).mean(dim = ["CLIMATE", "SSP", "BCM"])
dataset_ppd_gcm   = dataset_ppd.std(dim = ["GCM"]).mean(dim = ["CLIMATE", "SSP", "BCM"])

dataset_pp_ssp    = dataset_pp.std(dim = ["SSP"]).mean(dim = ["GCM", "CLIMATE", "BCM"])
dataset_t2m_ssp   = dataset_t2m.std(dim = ["SSP"]).mean(dim = ["GCM", "CLIMATE", "BCM"])
dataset_prsn_ssp  = dataset_prsn.std(dim = ["SSP"]).mean(dim = ["GCM", "CLIMATE", "BCM"])
dataset_ppd_ssp   = dataset_ppd.std(dim = ["SSP"]).mean(dim = ["GCM", "CLIMATE", "BCM"])

dataset_pp_bcm    = dataset_pp.std(dim = ["BCM"]).mean(dim = ["GCM", "SSP", "CLIMATE"])
dataset_t2m_bcm   = dataset_t2m.std(dim = ["BCM"]).mean(dim = ["GCM", "SSP", "CLIMATE"])
dataset_prsn_bcm  = dataset_prsn.std(dim = ["BCM"]).mean(dim = ["GCM", "SSP", "CLIMATE"])
dataset_ppd_bcm   = dataset_ppd.std(dim = ["BCM"]).mean(dim = ["GCM", "SSP", "CLIMATE"])

In [ ]:
# mean value for each catchment
averager = xe.SpatialAverager(dataset_pp,   basins.geometry, geom_dim_name="avg")

basins["PP_ensemble_std"]  = averager(dataset_pp_ensemble_std,    skipna=True).assign_coords(avg=xr.DataArray(basins.index, dims=("avg",))).values
basins["T2M_ensemble_std"] = averager(dataset_t2m_ensemble_std,   skipna=True).assign_coords(avg=xr.DataArray(basins.index, dims=("avg",))).values
basins["PP_ensemble_mean"]  = averager(dataset_pp_ensemble_mean,  skipna=True).assign_coords(avg=xr.DataArray(basins.index, dims=("avg",))).values
basins["T2M_ensemble_mean"] = averager(dataset_t2m_ensemble_mean, skipna=True).assign_coords(avg=xr.DataArray(basins.index, dims=("avg",))).values
basins["PP_ensemble_cv"]  = basins["PP_ensemble_std"] / basins["PP_ensemble_mean"]
basins["T2M_ensemble_cv"]  = basins["T2M_ensemble_std"] / basins["T2M_ensemble_mean"]

basins["PP_climate"]   = averager(dataset_pp_climate,   skipna=True).assign_coords(avg=xr.DataArray(basins.index, dims=("avg",))).values
basins["T2M_climate"]  = averager(dataset_t2m_climate,  skipna=True).assign_coords(avg=xr.DataArray(basins.index, dims=("avg",))).values
basins["PRSN_climate"] = averager(dataset_prsn_climate, skipna=True).assign_coords(avg=xr.DataArray(basins.index, dims=("avg",))).values
basins["PPD_climate"]  = averager(dataset_ppd_climate,  skipna=True).assign_coords(avg=xr.DataArray(basins.index, dims=("avg",))).values

basins["PP_gcm"]   = averager(dataset_pp_gcm,   skipna=True).assign_coords(avg=xr.DataArray(basins.index, dims=("avg",))).values
basins["T2M_gcm"]  = averager(dataset_t2m_gcm,  skipna=True).assign_coords(avg=xr.DataArray(basins.index, dims=("avg",))).values
basins["PRSN_gcm"] = averager(dataset_prsn_gcm, skipna=True).assign_coords(avg=xr.DataArray(basins.index, dims=("avg",))).values
basins["PPD_gcm"]  = averager(dataset_ppd_gcm,  skipna=True).assign_coords(avg=xr.DataArray(basins.index, dims=("avg",))).values

basins["PP_ssp"]   = averager(dataset_pp_ssp,   skipna=True).assign_coords(avg=xr.DataArray(basins.index, dims=("avg",))).values
basins["T2M_ssp"]  = averager(dataset_t2m_ssp,  skipna=True).assign_coords(avg=xr.DataArray(basins.index, dims=("avg",))).values
basins["PRSN_ssp"] = averager(dataset_prsn_ssp, skipna=True).assign_coords(avg=xr.DataArray(basins.index, dims=("avg",))).values
basins["PPD_ssp"]  = averager(dataset_ppd_ssp,  skipna=True).assign_coords(avg=xr.DataArray(basins.index, dims=("avg",))).values

basins["PP_bcm"]   = averager(dataset_pp_bcm,   skipna=True).assign_coords(avg=xr.DataArray(basins.index, dims=("avg",))).values
basins["T2M_bcm"]  = averager(dataset_t2m_bcm,  skipna=True).assign_coords(avg=xr.DataArray(basins.index, dims=("avg",))).values
basins["PRSN_bcm"] = averager(dataset_prsn_bcm, skipna=True).assign_coords(avg=xr.DataArray(basins.index, dims=("avg",))).values
basins["PPD_bcm"]  = averager(dataset_ppd_bcm,  skipna=True).assign_coords(avg=xr.DataArray(basins.index, dims=("avg",))).values

# major catchments
basins_m = basins.dropna(subset = ['Name'])


## Plot

In [ ]:
# basemap for background
geo_map = gpd.read_file("/home/rooda/Dropbox/ArcGIS/Chile/south_america.shp")
geo_map = geo_map[(geo_map.CC == "CI") | (geo_map.CC == "AR")]
geo_map = geo_map.dissolve(by='REGION')
geo_map["geometry"] = geo_map.simplify(0.01)

poly_gdf = shapely.geometry.Polygon([(-76, -55.7), (-76, -40.52), (-68.05, -40.52), (-68.05, -55.7), (-76, -55.8)])
poly_gdf = gpd.GeoDataFrame([1], geometry=[poly_gdf], crs=geo_map.crs)

geo_map = geo_map.clip(poly_gdf)

In [ ]:
# hydrological zone divides
geo_lines = gpd.read_file("/home/rooda/Dropbox/Patagonia/GIS South/Basins_Patagonia_ice_divides.shp")

lats = []
lons = []

for feature in geo_lines.geometry:
    if isinstance(feature, shapely.geometry.linestring.LineString):
        linestrings = [feature]
    elif isinstance(feature, shapely.geometry.multilinestring.MultiLineString):
        linestrings = feature.geoms
    else:
        continue
    for linestring in linestrings:
        x, y = linestring.xy
        lats = np.append(lats, y)
        lons = np.append(lons, x)
        lats = np.append(lats, None)
        lons = np.append(lons, None)
        
lat_coords = [-43.2, -45.95,  -46.4,  -47.55,  -49.2,   -50.5,   -52.0, -53.1, -54.8]
lon_coords = [-71.2, -71.7,   -74.5,  -71.7,   -72.2,   -72.3,   -72.1, -71.7, -68.9]
names      = ["PPY", "PCA", "NPI-W", "NPI-E", "SPI-N", "SPI-C", "SPI-S", "GCN", "CDI"]
names  = ['<b>'+x+'</b>' for x in names]

In [ ]:
fig = make_subplots(rows=3, cols=3, horizontal_spacing = 0.01, vertical_spacing = 0.02, column_widths = [0.36, 0.36, 0.31], shared_xaxes = True,
                    subplot_titles = ["Precipitation (PP) ensemble uncertainty","Temperature (T2M) ensemble uncertainty", "Individual annual uncertainty"],
                    specs=[[{"type": "scattergeo", "rowspan": 3}, {"type": "scattergeo", "rowspan": 3}, {"type": "histogram"}],
                           [          None,                                      None,                  {"type": "histogram"}],
                           [          None,                                      None,                  {"type": "histogram"}]])

## Basemap
for x in range(1,3):
    fig.add_trace(go.Choropleth(geojson = eval(geo_map['geometry'].to_json()),  locations = geo_map.index, z = geo_map['iso_num'], 
                            colorscale = ["#EAEAF2", "#EAEAF2"], showscale= False, marker_line_color ='white', marker_line_width=0.1), row=1, col=x)


# Precipitation mean change (a) -----------------------------------------------------------------------------------------------------------
fig.add_trace(go.Choropleth(geojson = eval(basins['geometry'].to_json()),  locations = basins.index, z = basins['PP_ensemble_std'], 
                            colorscale = ["#ccebc5", "#4eb3d3", "#034b8a"], marker_line_color ='white', marker_line_width=0.1, 
                            zmin = 500, zmax = 2500, colorbar=dict(len=0.45, x=0.24, y= 0.77, title='σ PP', ticksuffix = " mm", thickness=20)), row=1, col=1)
fig.add_trace(go.Choropleth(geojson = eval(basins['geometry'].to_json()),  locations = basins[basins.PP_ensemble_cv > 0.35].index, z = basins[basins.PP_ensemble_cv > 0.35].PP_ensemble_cv, 
                           colorscale = ["rgba(213,213,213,0)", "rgba(213,213,213,0)"], marker_line_color ='black', showscale= False, marker_line_width=0.15), row=1, col=1)
fig.add_annotation(text="a)", font=dict(size=16), x=0.005, y=0.995,  xref = "paper", yref = "paper", showarrow=False)



# Temperature mean change (b) -----------------------------------------------------------------------------------------------------------
fig.add_trace(go.Choropleth(geojson = eval(basins['geometry'].to_json()), locations = basins.index, z = basins['T2M_ensemble_std'], 
                            colorscale=[cs[0], cs[3],  cs[5]], marker_line_color='white', marker_line_width=0.1, 
                            zmin = 0.7, zmax = 2, colorbar=dict(len=0.45, x=0.61, y= 0.77, title='σ T2M', ticksuffix = " ºC", thickness=20, dtick= 0.5)), row=1, col=2)
fig.add_trace(go.Choropleth(geojson = eval(basins['geometry'].to_json()),  locations = basins[basins.T2M_ensemble_cv > 0.35].index, z = basins[basins.T2M_ensemble_cv > 0.35].PP_ensemble_cv, 
                           colorscale = ["rgba(213,213,213,0)", "rgba(213,213,213,0)"], marker_line_color ='black', showscale= False, marker_line_width=0.15), row=1, col=2)
fig.add_annotation(text="b)", font=dict(size=16), x=0.365, y=0.995,  xref = "paper", yref = "paper", showarrow=False)



# layout a) and b) ---------------------------------------------------------------------------------------------------------------------
for x in range(1,3):
    ## Add basin and hydrological zone names plus the hydro zone divides
    fig.add_trace(go.Scattergeo(lon = lons, lat = lats, mode = 'lines', line = dict(width = 0.7,color = 'black'),opacity = 0.5, showlegend = False),row=1, col=x)  
    fig.add_trace(go.Scattergeo(lon = lon_coords, lat=lat_coords, mode='text', text=names, textfont=dict(size=12, color = "rgba(0,0,0,0.5)"), showlegend = False),row=1, col=x)
    fig.add_scattergeo(geojson = eval(basins['geometry'].to_json()), locations = basins.index, text = basins['Name'], mode = 'text', showlegend = False,
                       textfont=dict(size=11, color = "rgba(0,0,0,0.3)"),row=1, col=x)

fig.update_geos(showframe = True, framewidth = 1,  framecolor = "black", lonaxis_range=[-76, -68], lataxis_range=[-55.8, -40.5], 
                bgcolor = "rgb(255,255,255)", showland = False, showcoastlines = False, showlakes = False)


# Solid precipitation spread (c) -----------------------------------------------------------------------------------------------------------
fig.add_trace(go.Violin(y = basins["PRSN_climate"],   marker_color = cl[2], name = "Climate", spanmode = "hard", opacity=0.7, points = False, showlegend = False), row=1, col=3)
fig.add_trace(go.Violin(y = basins["PRSN_ssp"],     marker_color = cs[5], name = "SSP", spanmode = "hard", opacity=0.7, points = False, showlegend = False), row=1, col=3)
fig.add_trace(go.Violin(y = basins["PRSN_gcm"],     marker_color = cl[0], name = "GCM", spanmode = "hard", opacity=0.7, points = False, showlegend = False), row=1, col=3)
fig.add_trace(go.Violin(y = basins["PRSN_bcm"],     marker_color = cl[5], name = "BCM", spanmode = "hard", opacity=0.7, points = False, showlegend = False), row=1, col=3)
fig.add_trace(go.Violin(y = basins_m["PRSN_climate"], marker_color = cl[2], name = "Climate", line_color= "rgba(255,255,255,0)", fillcolor= "rgba(255,255,255,0)",  points="all",  showlegend = False), row=1, col=3)
fig.add_trace(go.Violin(y = basins_m["PRSN_ssp"], marker_color = cs[5], name = "SSP", line_color= "rgba(255,255,255,0)", fillcolor= "rgba(255,255,255,0)",  points="all",  showlegend = False), row=1, col=3)
fig.add_trace(go.Violin(y = basins_m["PRSN_gcm"], marker_color = cl[0], name = "GCM", line_color= "rgba(255,255,255,0)", fillcolor= "rgba(255,255,255,0)",  points="all",  showlegend = False), row=1, col=3)
fig.add_trace(go.Violin(y = basins_m["PRSN_bcm"], marker_color = cl[5], name = "BCM", line_color= "rgba(255,255,255,0)", fillcolor= "rgba(255,255,255,0)",  points="all",  showlegend = False), row=1, col=3)
fig.update_traces(box_visible=True, width=0.7, meanline_visible=True, row = 1, col = 3)

fig.update_yaxes(title_text="σ Solid precpitation (mm)", side = "right", title_standoff = 2, row = 1, col = 3)
fig.update_yaxes(range = [0,800], tickangle = 0, row = 1, col = 3)
fig.add_annotation(text="c)", font=dict(size=16), x=-0.7, y=750, showarrow=False, row=1, col=3)


# Temperature spread (d) -----------------------------------------------------------------------------------------------------------
fig.add_trace(go.Violin(y = basins["T2M_climate"], marker_color = cl[2], spanmode = "hard", name = "Climate", opacity=0.7, points = False, showlegend = False), row=2, col=3)
fig.add_trace(go.Violin(y = basins["T2M_ssp"],     marker_color = cs[5], spanmode = "hard", name = "SSP", opacity=0.7, points = False, showlegend = False), row=2, col=3)
fig.add_trace(go.Violin(y = basins["T2M_gcm"],     marker_color = cl[0], spanmode = "hard", name = "GCM", opacity=0.7, points = False, showlegend = False), row=2, col=3)
fig.add_trace(go.Violin(y = basins["T2M_bcm"],     marker_color = cl[5], spanmode = "hard", name = "BCM", opacity=0.7, points = False, showlegend = False), row=2, col=3)
fig.add_trace(go.Violin(y = basins_m["T2M_climate"], marker_color = cl[2], name = "Climate", line_color= "rgba(255,255,255,0)", fillcolor= "rgba(255,255,255,0)",  points="all",  showlegend = False), row=2, col=3)
fig.add_trace(go.Violin(y = basins_m["T2M_ssp"], marker_color = cs[5], name = "SSP", line_color= "rgba(255,255,255,0)", fillcolor= "rgba(255,255,255,0)",  points="all",  showlegend = False), row=2, col=3)
fig.add_trace(go.Violin(y = basins_m["T2M_gcm"], marker_color = cl[0], name = "GCM", line_color= "rgba(255,255,255,0)", fillcolor= "rgba(255,255,255,0)",  points="all",  showlegend = False), row=2, col=3)
fig.add_trace(go.Violin(y = basins_m["T2M_bcm"], marker_color = cl[5], name = "BCM", line_color= "rgba(255,255,255,0)", fillcolor= "rgba(255,255,255,0)",  points="all",  showlegend = False), row=2, col=3)
fig.update_traces(box_visible=True, width=0.7, meanline_visible=True, row = 2, col = 3)

fig.update_yaxes(title_text="σ Temperature (ºC)", side = "right", title_standoff = 5, row = 2, col = 3)
fig.update_yaxes(range = [0, 2], tickangle = 0, row = 2, col = 3)
fig.add_annotation(text="d)", font=dict(size=16), x=-0.7, y=1.9, showarrow=False, row=2, col=3)


# Positive degree-day sum spread (e) -----------------------------------------------------------------------------------------------------------
fig.add_trace(go.Violin(y = basins["PPD_climate"], marker_color = cl[2], spanmode = "hard", name = "Climate", opacity=0.7, points = False, showlegend = False), row=3, col=3)
fig.add_trace(go.Violin(y = basins["PPD_ssp"],     marker_color = cs[5], spanmode = "hard", name = "SSP", opacity=0.7, points = False, showlegend = False), row=3, col=3)
fig.add_trace(go.Violin(y = basins["PPD_gcm"],     marker_color = cl[0], spanmode = "hard", name = "GCM", opacity=0.7, points = False, showlegend = False), row=3, col=3)
fig.add_trace(go.Violin(y = basins["PPD_bcm"],     marker_color = cl[5], spanmode = "hard", name = "BCM", opacity=0.7, points = False, showlegend = False), row=3, col=3)
fig.add_trace(go.Violin(y = basins_m["PPD_climate"], marker_color = cl[2], name = "Climate", line_color= "rgba(255,255,255,0)", fillcolor= "rgba(255,255,255,0)",  points="all",  showlegend = False), row=3, col=3)
fig.add_trace(go.Violin(y = basins_m["PPD_ssp"], marker_color = cs[5], name = "SSP", line_color= "rgba(255,255,255,0)", fillcolor= "rgba(255,255,255,0)",  points="all",  showlegend = False), row=3, col=3)
fig.add_trace(go.Violin(y = basins_m["PPD_gcm"], marker_color = cl[0], name = "GCM", line_color= "rgba(255,255,255,0)", fillcolor= "rgba(255,255,255,0)",  points="all",  showlegend = False), row=3, col=3)
fig.add_trace(go.Violin(y = basins_m["PPD_bcm"], marker_color = cl[5], name = "BCM", line_color= "rgba(255,255,255,0)", fillcolor= "rgba(255,255,255,0)",  points="all",  showlegend = False), row=3, col=3)
fig.update_traces(box_visible=True, width=0.7, meanline_visible=True, row = 3, col = 3)

fig.update_yaxes(title_text="σ Positive degree-day sum (ºC)", side = "right", title_standoff = 5, row = 3, col = 3)
fig.update_yaxes(range = [0,500], tickangle = 0, row = 3, col = 3)
fig.add_annotation(text="e)", font=dict(size=16), x=-0.7, y=470, showarrow=False, row=3, col=3)
fig.add_annotation(text="Main catchments", font=dict(size=14), ax=35, x=0.55, y=300, showarrow=True, row=3, col=3)


# layout c, d and e
fig.update_xaxes(griddash = "dot", gridcolor = "rgba(255,255,255,0.8)", zeroline=False, showline = True, linecolor = 'black', linewidth = 1, ticks="outside", mirror=True)
fig.update_yaxes(griddash = "dot", gridcolor = "rgba(255,255,255,0.8)", zeroline=True,  showline = True, linecolor = 'black', linewidth = 1, ticks="outside", mirror=True)

# general 
fig.update_layout(autosize = False, template = "seaborn", width = 1000, height = 670, margin = dict(l=10, r=5, b=5, t=30, pad=0, autoexpand=True))

fig.write_image("/home/rooda/Dropbox/Patagonia/MS2 Results/Figure_7_climate_uncertainty.png", scale=4)
fig.show()

In [ ]:
# % of catchmment area: precipitation > 1000
basins[basins["PP_ensemble_std"] > 1000].basin_area.sum() / basins.basin_area.sum()

In [ ]:
# % of glacier area: precipitation > 1000
basins[basins["PP_ensemble_std"] > 1000].RGI6_area.sum() / basins.RGI6_area.sum()

In [ ]:
# % of catchmment area: t2m > 1ºC
basins[basins["T2M_ensemble_std"] > 1].basin_area.sum() / basins.basin_area.sum()

In [ ]:
# % of glacier area: t2m > 1ºC
basins[basins["T2M_ensemble_std"] > 1].RGI6_area.sum() / basins.RGI6_area.sum()

In [ ]:
(basins["T2M_ensemble_cv"] > 0.35).sum()